In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import time
from sklearn.preprocessing import LabelEncoder

In [87]:
train_voc = pd.read_csv('../train/train_voc.csv',encoding = 'utf-8')
test_voc = pd.read_csv('../test/test_voc.csv',encoding = 'utf-8')

F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
train_voc["start_datetime"] = pd.to_datetime(train_voc['start_datetime'])
train_voc["hour"] = train_voc['start_datetime'].dt.hour
train_voc["day"] = train_voc['start_datetime'].dt.day
train_voc['month'] = train_voc['start_datetime'].dt.month
train_voc['start_datetime_shift'] = train_voc['start_datetime'].shift(-1)

In [89]:
test_voc["start_datetime"] = pd.to_datetime(test_voc['start_datetime'])
test_voc["hour"] = test_voc['start_datetime'].dt.hour
test_voc["day"] = test_voc['start_datetime'].dt.day
test_voc['month'] = test_voc['start_datetime'].dt.month
test_voc['start_datetime_shift'] = test_voc['start_datetime'].shift(-1)

In [90]:
train_voc_3 = train_voc[train_voc['month']==3]

In [91]:
df_voc = pd.concat([train_voc_3,test_voc], axis = 0).reset_index(drop = True)

df_voc['voc_city_null'] = df_voc['city_name'].isnull().astype(np.int64)

df_voc = pd.concat([df_voc,pd.get_dummies(df_voc['calltype_id'],prefix = 'call_type')],axis = 1)

# phone =df_voc['phone_no_m'].drop_duplicates().tolist()
df_voc[['city_name','county_name']] = df_voc[['city_name','county_name']].fillna('缺失')

In [92]:
phone_no_m = df_voc[['phone_no_m']].copy()
phone_no_m = phone_no_m.drop_duplicates(subset = ['phone_no_m'], keep = 'last')

In [93]:
#对话人数和对话次数
temp = df_voc.groupby('phone_no_m')['opposite_no_m'].agg(
    voc_opposite_count = 'count',
    voc_opposite_unique = 'nunique',
    voc_opposite_mean = lambda x : x.value_counts().mean(), 
    voc_opposite_max = lambda x: x.value_counts().max())
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')

In [94]:
# 主叫通话
df_call = df_voc[df_voc['calltype_id'] == 1].copy()
temp = df_call.groupby('phone_no_m')['imei_m'].agg(
    voc_calltype_1 = 'count',
    voc_imeis = 'nunique')
phone_no_m = phone_no_m.merge(temp, on="phone_no_m", how="left")
phone_no_m["voc_calltype1_per_oppo"] = phone_no_m["voc_calltype_1"] / (phone_no_m["voc_opposite_count"])
temp = df_call.groupby("phone_no_m")["city_name"].agg(voc_city_unique="nunique")
phone_no_m = phone_no_m.merge(temp, on="phone_no_m", how="left")
temp = df_call.groupby("phone_no_m")["county_name"].agg(voc_county_unique="nunique")
phone_no_m = phone_no_m.merge(temp, on="phone_no_m", how="left")

In [95]:
# 通话时长
voc_month_count = df_voc.groupby('phone_no_m')['month'].agg(lambda x : len(x.value_counts())).values
temp = df_voc.groupby('phone_no_m')['call_dur'].agg(
    call_dur_mean = 'mean',
    call_dur_min = 'min',
    call_dur_sum = 'sum',
    call_dur_max = 'max',
    call_dur_std = 'std',
#     call_dur_var = 'var',
#     call_dur_median = 'median'
)
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')
phone_no_m['call_dur_per_month'] = phone_no_m['call_dur_sum'] / voc_month_count

In [96]:
#和固定通话者的对话统计
tmp = df_voc.groupby(["phone_no_m","opposite_no_m"])["call_dur"].agg(count="count", sum="sum")
phone2opposite = tmp.groupby("phone_no_m")["count"].agg(phone2opposite_mean="mean", phone2opposite_median="median", phone2opposite_max="max")
phone_no_m = phone_no_m.merge(phone2opposite, on="phone_no_m", how="left")
phone2opposite = tmp.groupby("phone_no_m")["sum"].agg(phone2oppo_sum_mean="mean", phone2oppo_sum_median="median", phone2oppo_sum_max="max")
phone_no_m = phone_no_m.merge(phone2opposite, on="phone_no_m", how="left")


In [97]:
## 通话类型计数
temp = df_voc.groupby('phone_no_m')['calltype_id'].agg(
    call_type1_count = lambda x : (x == 1).sum(),
    call_type2_count = lambda x: (x==2).sum(),
    call_type3_count = lambda x : (x==3).sum(),
    call_called_rate = lambda x : ((x == 1).sum() + 1)/((x==2).sum() + 1)
)
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')

In [98]:
## 通话位置
temp = df_voc.groupby('phone_no_m')['city_name'].agg(voc_city = lambda x: x.value_counts().index[0],
                                                    callout_city_unique = 'nunique')
phone_no_m = phone_no_m.merge(temp, on="phone_no_m", how="left")
temp = df_voc.groupby('phone_no_m')['county_name'].agg(voc_county = lambda x: x.value_counts().index[0],
                                                      callout_county_unique = 'nunique')
phone_no_m = phone_no_m.merge(temp, on="phone_no_m", how="left")

In [99]:
## 通话时间习惯
temp = df_voc.groupby('phone_no_m')['hour'].agg(voc_hour_mode = lambda x:stats.mode(x)[0][0], voc_hour_mode_count = lambda x:stats.mode(x)[1][0], voc_hour_count = 'nunique', voc_hour_kurt = lambda x:x.kurt(), voc_hour_skew = lambda x:x.skew())
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')
temp = df_voc.groupby('phone_no_m')['day'].agg(voc_day_mode = lambda x:stats.mode(x)[0][0], voc_day_mode_count = lambda x:stats.mode(x)[1][0], voc_day_count = 'nunique', voc_day_kurt = lambda x:x.kurt(), voc_day_skew = lambda x:x.skew())
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')

In [100]:
temp = pd.DataFrame(df_voc.groupby('phone_no_m').apply(lambda x :np.min((x['start_datetime_shift'] - x['start_datetime']).apply(lambda x:x.seconds))),columns = ['voc_min_sep'])
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')
# temp = pd.DataFrame(df_voc.groupby('phone_no_m').apply(lambda x : np.max((x['start_datetime_shift'] - x['start_datetime']).apply(lambda x:x.seconds))))
# phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')
temp = pd.DataFrame(df_voc.groupby('phone_no_m').apply(lambda x : np.min((x['start_datetime_shift'] - x['start_datetime']).apply(lambda x : x.seconds) < 60)),columns=['voc_min_sep_low'])
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')
temp = pd.DataFrame(df_voc.groupby('phone_no_m').apply(lambda x : np.min((x['start_datetime_shift'] - x['start_datetime']).apply(lambda x : x.seconds) > 1800)),columns=['voc_min_sep_high'])
phone_no_m = phone_no_m.merge(temp, on = 'phone_no_m', how = 'left')






In [101]:
new_df = phone_no_m

In [102]:
# 对城市和区LabelEncoder
# le = LabelEncoder()
# for feature in ['voc_city','voc_county']:
#     new_df[feature] = le.fit_transform(new_df[feature])
# 对城市和区one_hot encoding
new_df = pd.concat([new_df,pd.get_dummies(new_df[['voc_city','voc_county']])],axis=1)
new_df.drop(['voc_city','voc_county'], axis = 1, inplace = True)

In [105]:
new_df.to_csv('./voc_submit.csv',encoding='utf-8',index=None)

In [103]:
new_df

,phone_no_m,voc_opposite_count,voc_opposite_unique,voc_opposite_mean,voc_opposite_max,voc_calltype_1,voc_imeis,voc_calltype1_per_oppo,voc_city_unique,voc_county_unique,...,voc_county_马尔康县,voc_county_马边彝族自治县,voc_county_高县,voc_county_高坪区,voc_county_高新区,voc_county_高新南区,voc_county_高新西区,voc_county_黑水县,voc_county_龙泉驿区,voc_county_龙马潭区
0,e74091a7f55e5df0b8dfc10538025b3648c16cc1491434...,1,1,1.000000,1,1.0,1.0,1.000000,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,39456e1e101705250790d5f2d4409f31cf185f8da3e6a4...,1,1,1.000000,1,1.0,1.0,1.000000,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,97577045f480a3d6c6c6f0ddb74ce02f880939dd8df5b0...,2,2,1.000000,1,2.0,1.0,1.000000,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,7fe7ab2ef4be21b9cfd1c402fc1fdf4dc1359227d87ce8...,1,1,1.000000,1,1.0,1.0,1.000000,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,514cb3f69a19736137f6815af247cc661add1b467b1370...,4,2,2.000000,3,4.0,1.0,1.000000,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6783,8b1da0307ad4ce1012c471b51f3ef9437fb4d380e4f25a...,134,32,4.187500,69,84.0,1.0,0.626866,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
6784,fc43c2c390e2cb935faf1a82ed521bcb2676d82af561fa...,96,48,2.000000,17,60.0,1.0,0.625000,2.0,4.0,...,0,0,0,0,0,0,0,0,0,0
6785,b44c35fc9fb3884c55963cb40938a97b77debc5f280d61...,35,17,2.058824,8,10.0,1.0,0.285714,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
6786,82faeb8be1e5fae1c5efd008983b9d90218a8a12a23c6f...,133,22,6.045455,44,78.0,1.0,0.586466,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0
